# Transient absorption case study of the chromophoric systems rc and rcg

## part 2 rcg target

## part 2 rcg target

van Stokkum IHM, Wohlmuth C, Würthner F, Williams RM (2022) Energy transfer in supramolecular calix[4]arene—Perylene bisimide dye light harvesting building blocks: Resolving loss processes with simultaneous target analysis. Journal of Photochemistry and Photobiology 12:100154. doi:https://doi.org/10.1016/j.jpap.2022.100154

# Inspect experimental data


The guidance spectra used here have been prepared outside of pyglotaran. In the rc notebook (part 1) it is described how guidance spectra can be created.

In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = {
    "tas_rcg_dcm": "experiment_data/Krg_grga.ascii",
    "guide_rcg_r1": "experiment_data/Krg_grgc.ascii",
    "guide_rcg_r2": "experiment_data/Krg_grgd.ascii",
    "guide_rcg_r3": "experiment_data/Krg_grge.ascii",
    "guide_rcg_r4": "experiment_data/Krg_grgf.ascii",
    "guide_rcg_g": "experiment_data/Krg_grgg.ascii",
}

plot_data_overview(experiment_data["tas_rcg_dcm"], linlog=True, linthresh=1);

# Create a project and import the data

In [ ]:
from glotaran.project import Project

project = Project.open("")
project.import_data(experiment_data)

# Model and Parameter definition


In [ ]:
project.show_model_definition("target_rcg_refine")

![figure_rcg][def]

[def]: ./formula.png

In [ ]:
project.show_parameters_definition("target_rcg_refine")

# Optimization


1.0408e+04 nev 1
new
8.9719e+03 nev 1 !


In [ ]:
result = project.optimize(
    model_name="target_rcg_refine",
    parameters_name="target_rcg_refine",
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=1,
)

## Inspect fit quality


In [ ]:
result

## Plot fitted traces


In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

wavelengths = select_plot_wavelengths(
    result.data["tas_rcg_dcm"], equidistant_wavelengths=False, axes_shape=[6, 5]
)
fig3tr, axes = plot_fitted_traces(result, wavelengths, axes_shape=[6, 5], linlog=True, linthresh=1)

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas

myFRLcolors = [
    "tab:grey",
    "tab:orange",
    "r",
    "tab:purple",
    "g",
    "k",
    "b",
    "m",
    "c",
    "y",
    "tab:brown",
]
custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(result.data["tas_rcg_dcm"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

# Plot result for interpretation

## Overview


In [ ]:
# from pyglotaran_extras import plot_overview

# plot_overview(result, linlog=True, linthresh=1, figure_only=False);

## Fit of the guidance SADS


In [ ]:
from pyglotaran_extras.plotting.style import PlotStyle

# fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.set_prop_cycle(PlotStyle().data_cycler_solid_dashed)
for data_set_suffix in ["r1", "r2", "r3", "r4", "g"]:
    dataset_name = f"guide_rcg_{data_set_suffix}"
    result.data[dataset_name].data.plot(label=f"{dataset_name} data", ax=ax)
    result.data[dataset_name].fitted_data.plot(label=f"{dataset_name} fitted data", ax=ax)

ax.legend()
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("SADS (a.u.)")
ax.set_title("Fit of the guidance SADS")
ax.axhline(0, color="k", linewidth=1)
# axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction",fontsize=16)

## Plot of the concentrations

In [ ]:
# NB here one needs to select the species_concentration of a single wavelength
import xarray as xr
from cycler import cycler

# fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
myFRLcolors = ["tab:grey", "tab:orange", "r", "tab:purple", "g", "k"]
center_λ = 600


def plot_concentrations_compare(dataset=xr.Dataset, linestyle=str):
    plot_concentrations(
        dataset,
        ax=ax,
        center_λ=center_λ,
        cycler=cycler(color=myFRLcolors, linestyle=[linestyle] * 6),
        linlog=True,
        linthresh=1,
    )


plot_concentrations_compare(result.data["tas_rcg_dcm"], "-")
# plot_concentrations_compare(result.data["tas_gcrcg_dcm"], "--")
# plot_concentrations_compare(result.data["tas_rcgcr_dcm"], ":")


ax.set_xlabel("Time (ps)")
ax.set_ylabel("Population")
ax.legend(result.data["tas_rcg_dcm"].species.values);

## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

irfas_plot_wavelength = 420

fig, axes = plot_coherent_artifact(
    result.data["tas_rcg_dcm"],
    time_range=(-0.3, 0.3),
    spectral=irfas_plot_wavelength,
    figsize=(10, 3),
)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].get_legend().remove()
# axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
# axes[1].set_ylabel("SADS (mOD)")
# axes[1].set_title("SADS")
# axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(0.01, 0.91), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.01, 0.91), xycoords="axes fraction", fontsize=16)

## Amplitude matrix of the rcg target analysis

In [ ]:
from pyglotaran_extras.inspect import show_a_matrixes

show_a_matrixes(result)

## Target analysis using a kinetic scheme without the Radical Pair state (noRP)

In [ ]:
result_noRP = project.optimize(
    model_name="target_noRP_rcg_refine",
    parameters_name="target_noRP_rcg_refine",
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=7,
)

In [ ]:
result

In [ ]:
result_noRP

## Comparison of the first left and right singular vectors of the residual matrices of the two models (RP vs noRP)

In [ ]:
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_svd_panel(result_dataset, result2_dataset, linlog=True):
    fig, axes = plt.subplots(2, 2, figsize=(10, 7), squeeze=False)
    plot_lsv_residual(result_dataset, axes[1][0], indices=[0], linlog=linlog, linthresh=1)
    plot_rsv_residual(result_dataset, axes[1][1], indices=[0])
    plot_lsv_residual(result2_dataset, axes[0][0], indices=[0], linlog=linlog, linthresh=1)
    plot_rsv_residual(result2_dataset, axes[0][1], indices=[0])

    return fig, axes


fig, axes = plot_svd_panel(result.data["tas_rcg_dcm"], result_noRP.data["tas_rcg_dcm"])
axes[0][0].get_legend().remove()
axes[0][0].set_xlabel("Time (ps)")
axes[0][0].set_xlabel("")
axes[0][0].set_ylabel("")
axes[0][1].get_legend().remove()
axes[0][1].set_xlabel("Wavelength (nm)")
axes[0][1].set_xlabel("")
axes[0][1].set_ylabel("")
axes[1][0].get_legend().remove()
axes[1][0].set_xlabel("Time (ps)")
axes[1][0].set_ylabel("")
axes[1][1].get_legend().remove()
axes[1][1].set_xlabel("Wavelength (nm)")
axes[1][1].set_ylabel("")
axes[1][0].set_title("residual with RP 1st LSV")
axes[1][1].set_title("residual with RP 1st RSV")
axes[0][0].set_title("residual no RP 1st LSV")
axes[0][1].set_title("residual no RP 1st RSV")
axes[0][0].annotate("A", xy=(0.01, 0.92), xycoords="axes fraction", fontsize=16)
axes[0][1].annotate("B", xy=(0.01, 0.92), xycoords="axes fraction", fontsize=16)
axes[1][0].annotate("C", xy=(0.01, 0.92), xycoords="axes fraction", fontsize=16)
axes[1][1].annotate("D", xy=(0.01, 0.92), xycoords="axes fraction", fontsize=16)
for ax in axes[0:3, 0:3].flatten():
    ax.axhline(0, color="k", linewidth=1)
fig.tight_layout()